# LangChain + Klavis AI Integration

This tutorial demonstrates how to build AI agents using LangChain with Klavis Strata MCP servers for enhanced functionality.


## Prerequisites

Before we begin, you'll need:

- **OpenAI API key** - Get at [openai.com](https://openai.com/)
- **Klavis API key** - Get at [klavis.ai](https://klavis.ai/)


In [ ]:
# Install the required packages
%pip install -q klavis python-dotenv langchain-mcp-adapters langgraph langchain-openai


In [ ]:
import os
import webbrowser
from klavis import Klavis
from klavis.types import McpServerName
from langchain_openai import ChatOpenAI
from langchain_mcp_adapters.client import MultiServerMCPClient
from langgraph.prebuilt import create_react_agent

# Set environment variables
os.environ["OPENAI_API_KEY"] = "YOUR_OPENAI_API_KEY"  # Replace with your actual OpenAI API key
os.environ["KLAVIS_API_KEY"] = "YOUR_KLAVIS_API_KEY"  # Replace with your actual Klavis API key


## Step 1: Create Strata MCP Server

Create a unified MCP server that combines multiple services (Gmail and YouTube) for enhanced agent capabilities.


In [ ]:
klavis_client = Klavis(api_key=os.getenv("KLAVIS_API_KEY"))

# Create a Strata MCP server with Gmail and YouTube integrations
response = klavis_client.mcp_server.create_strata_server(
    user_id="demo_user",
    servers=[McpServerName.GMAIL, McpServerName.YOUTUBE],
)

print(f"🔗 Strata MCP server created at: {response.strata_server_url}")

# Handle OAuth authorization if needed
if response.oauth_urls:
    for server_name, oauth_url in response.oauth_urls.items():
        webbrowser.open(oauth_url)
        print(f"🔐 Opening OAuth authorization for {server_name}: {oauth_url}")
        input(f"Press Enter after completing {server_name} OAuth authorization...")


## Step 2: Create LangChain Agent with MCP Tools

Set up the LangChain agent with tools from the Strata MCP server.


In [ ]:
# Create MCP client
mcp_client = MultiServerMCPClient({
    "strata": {
        "transport": "streamable_http",
        "url": response.strata_server_url,
    }
})

# Get all available tools from Strata
tools = await mcp_client.get_tools()

# Setup LLM
llm = ChatOpenAI(model="gpt-4o-mini", api_key=os.getenv("OPENAI_API_KEY"))

# Create LangChain agent with MCP tools
agent = create_react_agent(
    model=llm,
    tools=tools,
    prompt="You are a helpful assistant that can use MCP tools.",
)

print("🤖 LangChain agent created with MCP tools!")


## Step 3: Run the Agent

Use the agent to summarize a YouTube video and send the summary via email.


In [ ]:
# Configure your preferences
my_email = "your-email@example.com"  # Replace with your email
youtube_video_url = "https://youtu.be/yebNIHKAC4A?si=1Rz_ZsiVRz0YfOR7"  # Replace with your favorite video

# Invoke the agent
result = await agent.ainvoke({
    "messages": [{
        "role": "user", 
        "content": f"summarize this video - {youtube_video_url} and send the summary to my email {my_email}"
    }],
})

# Print the final AI response
print("\n✅ Result:")
print(result["messages"][-1].content)


## Summary

🎉 Congratulations! You've successfully created a LangChain agent that can:

1. **Summarize YouTube videos** using the YouTube MCP server
2. **Send emails** using the Gmail MCP server  
3. **Coordinate multiple services** through Klavis Strata MCP integration

This demonstrates the power of combining LangChain's agent framework with Klavis MCP servers for building sophisticated AI workflows that can interact with multiple external services seamlessly.
